# Python for Data Science Teaching Session 4: Data Transformation

## Introduction

### Session Objectives

- Combining datasets and filtering joins
- Dealing with missing values
- Reshaping data
- Dummifying categorical variables

### Coming Full Circle

In this session, we will be returning to Pandas, to learn how we can use it for transforming data. We will start by importing the package as well as another we'll need for loading the datasets we'll be working with.

In [1]:
# Import packages
from urllib.request import urlopen

import pandas as pd

## Combining Datasets

In this section, we will be working with a collection of datasets related to airline data for planes departing NYC in 2013. The code below loads in a selection of dataframes from this course's GitHub repository, saving them in a dictionary, `data`.

In [2]:
base_url = ('https://raw.githubusercontent.com/warwickdatasciencesociety/'
            'python-for-data-science/master/session-four/data/nycflights13/')
# Collect list of file names
with urlopen(base_url + 'names.txt') as f:
    names = f.read().decode('utf-8').split()
    
# Import data
data = {}
for n in names:
    data[n] = pd.read_csv(base_url + n + '.csv')

We can see a list of the dataframes and view any particular one using standard dictionary techniques.

In [3]:
# Dataset list
print(*data.keys(), sep=' | ')

airlines | airports_loc | airports_tz | flights_q12 | flights_q34 | planes


In [16]:
# Airlines dataset
data['airports_loc']

,faa,name,lat,lon,alt,dst
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,A
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,A
2,06C,Schaumburg Regional,41.989341,-88.101243,801,A
3,06N,Randall Airport,41.431912,-74.391561,523,A
4,09J,Jekyll Island Airport,31.074472,-81.427778,11,A
...,...,...,...,...,...,...
1453,ZUN,Black Rock,35.083228,-108.791778,6454,A
1454,ZVE,New Haven Rail Station,41.298669,-72.925992,7,A
1455,ZWI,Wilmington Amtrak Station,39.736667,-75.551667,0,A
1456,ZWU,Washington Union Station,38.897460,-77.006430,76,A


### Concatenation

We will begin by learning how to concatenate data. Concatenation is used when we have two dataframes that either share the same columns or same rows. We can then join along the non-matching axis to obtain a single dataframe. This is done using the `pd.concat()` function which accepts a list-like object of dataframes to combine as well as the axis to combine over.

We start by concatenating rows. In this case, we use `axis=0`. This is the default, so we don't need to specify it, but it is helpful to people reading our code later (which may even be us) to include it.

We can do this for the two flights dataset, which contain the same columns but for different quarters. Before combining, it is best practice to check that the columns do indeed match up (though order doesn't matter).

In [11]:
# Check both flight datasets have the same columns
set(data['flights_q12'].columns) == set(data['flights_q34'].columns)

True

In [13]:
# Combine the flights datasets
data['flights'] = pd.concat(
    (data['flights_q12'], data['flights_q34']),
    axis=0
)
data['flights'].sample(5)

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
158021,2013,6,22,1110.0,1125,-15.0,1211.0,1231,-20.0,B6,2380,N348JB,EWR,BOS,43.0,200,11,25,2013-06-22T15:00:00Z
81375,2013,12,28,1603.0,1610,-7.0,1727.0,1801,-34.0,9E,2900,N925XJ,JFK,BNA,124.0,765,16,10,2013-12-28T21:00:00Z
43752,2013,2,20,657.0,700,-3.0,1012.0,1025,-13.0,WN,20,N936WN,EWR,HOU,230.0,1411,7,0,2013-02-20T12:00:00Z
47445,2013,2,24,906.0,910,-4.0,1033.0,1053,-20.0,B6,885,N266JB,JFK,RDU,69.0,427,9,10,2013-02-24T14:00:00Z
62608,2013,3,12,1445.0,1450,-5.0,1603.0,1627,-24.0,9E,4357,N8783E,JFK,ORF,56.0,290,14,50,2013-03-12T18:00:00Z


> Note, there is no limit to how many dataframes you can pass into `pd.concat`.

Likewise, we use `axis=1` to be combine columns. We can use this to combine the two datasets giving location and timezone information for airports. In this case, both datasets contain the columns `faa` and `name` so we will remove these from the latter. Like before, we check the indices match before concatenating.

In [17]:
# Check both airport datasets have the same rows
set(data['airports_loc'].index) == set(data['airports_tz'].index)

True

In [18]:
# Combine the airports datasets
data['airports'] = pd.concat(
    (data['airports_loc'], data['airports_tz'].drop(['faa', 'name'], 1)),
    axis=1
)
data['airports'].head()

,faa,name,lat,lon,alt,dst,tz,tzone
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,A,-5,America/New_York
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,A,-6,America/Chicago
2,06C,Schaumburg Regional,41.989341,-88.101243,801,A,-6,America/Chicago
3,06N,Randall Airport,41.431912,-74.391561,523,A,-5,America/New_York
4,09J,Jekyll Island Airport,31.074472,-81.427778,11,A,-5,America/New_York


There are a few edge-cases we might want to consider.

What if we don't have matching rows/columns? In this case, the default behaviour is to use missing values to fill any gaps. If however, we specify `join='inner'`, any rows/columns not found in all dataframes will be removed.

What if we have duplicate columns or indexes? This can result in strange behaviour as this is not the intended use case of `concat`.

If you find yourself in either of the two situations above, you probably want a join, which we will learn about now.

> **Further Reading**
>
> The `pd.concat()` function has a large number of optional parameters for handling special cases and applying post-processing steps. Read [the documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html) to learn more.

### Merges and Joins

Many programming languages and database management systems facilitate the joining of data. This is used when we have two columns in two dataframes that represent the same variable and we intend to use this to combine the data into one table. This differs from concatenation in that there is no expectation that the two columns columns used in the join share the same values or that there are no duplicates. In fact, joins are meant for exactly these two cases.

We will create two dummy datasets to introduce the concept of joins. These relate to the employees in a fictional company and the sales they made. Notice how Cat made no sales, and sales were made by Dan despite him no longer being an employee.

In [19]:
# Create example datasets
employee = pd.DataFrame({
    'name': ["Ann", "Bob", "Cat"],
    'years_exp': [4, 2, 1]
})
sale = pd.DataFrame({
    'name': ["Ann", "Bob", "Dan", "Ann", "Dan", "Bob", "Ann"],
    'day': ["Mon", "Mon", "Tue", "Wed", "Thu", "Thu", "Fri"],
    'value': [5, 7, 3, 8, 3, 4, 8]
})
display(employee)
display(sale)

,name,years_exp
0,Ann,4
1,Bob,2
2,Cat,1


,name,day,value
0,Ann,Mon,5
1,Bob,Mon,7
2,Dan,Tue,3
3,Ann,Wed,8
4,Dan,Thu,3
5,Bob,Thu,4
6,Ann,Fri,8


The column used to join the two dataframes is referred to as the key, in this case it is the employee name. There are four common types of join that differ in how they handle values that don't appear in one of the key columns:

- `inner` (default): keep only rows that have matching key values in both dataframes
- `left`: keep only rows for every key value in the left-hand dataframe
- `right`: keep only rows for every key value in the right-hand dataframe
- `outer`: keep all rows

All joins but the inner join can introduce missing values, as we will see in a moment. When we talk about the left/right-hand tables, we are referring to which comes first (left) and second (right) in the function call. Confusingly, Pandas uses the function `merge` to perform what is typically called a `join`. The `join` function performs a similar function which we will see later.

The simplest join is the inner join, only keeping key values that appeared in both tables. We perform an inner join as so.

In [20]:
# Perform an inner join
pd.merge(employee, sale, how='inner', on='name')

,name,years_exp,day,value
0,Ann,4,Mon,5
1,Ann,4,Wed,8
2,Ann,4,Fri,8
3,Bob,2,Mon,7
4,Bob,2,Thu,4


A left join, gives precedence to the left-hand dataframe, keeping all values of its key column. This might involve filling unknown values with `np.nan`s.

In [12]:
# Perform a left join
pd.merge(employee, sale, how='left', on='name')

,name,years_exp,day,value
0,Ann,4,Mon,5.0
1,Ann,4,Wed,8.0
2,Ann,4,Fri,8.0
3,Bob,2,Mon,7.0
4,Bob,2,Thu,4.0
5,Cat,1,NaN,NaN


A right join has the reverse effect.

In [13]:
# Perform a right join
pd.merge(employee, sale, how='right', on='name')

,name,years_exp,day,value
0,Ann,4.0,Mon,5
1,Bob,2.0,Mon,7
2,Dan,NaN,Tue,3
3,Ann,4.0,Wed,8
4,Dan,NaN,Thu,3
5,Bob,2.0,Thu,4
6,Ann,4.0,Fri,8


An outer join (also called a full/full outer join) is the most lenient, including any key value that appears in either table.

In [14]:
# Perform an outer join
pd.merge(employee, sale, how='outer', on='name')

,name,years_exp,day,value
0,Ann,4.0,Mon,5.0
1,Ann,4.0,Wed,8.0
2,Ann,4.0,Fri,8.0
3,Bob,2.0,Mon,7.0
4,Bob,2.0,Thu,4.0
5,Cat,1.0,NaN,NaN
6,Dan,NaN,Tue,3.0
7,Dan,NaN,Thu,3.0


In this session, we don't have to time to explain why we store data in a format that requires joining or why we would choose one join type over another. We will instead discuss this in the project sessions with a practical example.

In the case when our key columns have different names, we use `left_on` and `right_on` instead of the single `on`.

A join in Pandas is used when we want to join the index of one dataframe with a column or index from another. Essentially, it is a more restrictive version of `merge` (which can also use indices—check [the docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)), enjoying more efficiency in return. Since `merge` encompasses this functionality, we will look at one example and then move on. Feel free to read up more on `join` if you are keen to use best practice.

In [21]:
# Convert name column to an index
employee_index = employee.set_index('name')
employee_index

,years_exp
name,
Ann,4
Bob,2
Cat,1


In [16]:
# Perform a join using the index
sale.join(employee_index, on='name')  # default is 'left'

,name,day,value,years_exp
0,Ann,Mon,5,4.0
1,Bob,Mon,7,2.0
2,Dan,Tue,3,NaN
3,Ann,Wed,8,4.0
4,Dan,Thu,3,NaN
5,Bob,Thu,4,2.0
6,Ann,Fri,8,4.0


> **Going Beyond**
>
> We've only just scratched the surface of what Pandas joins and merges can do. Here are a few extra ideas for you to play with:
>
> - We can join by a key composed of multiple columns by using a list of column names for `on`, `left_on`, `right_on`
> - We can also write `pd.merge(df1, df2, ...)` as `df1.merge(df2, ...)`
> - There is another type of join called a cross-join used for creating all combinations of keys
> - When dataframes have overlapping column names, suffixes are applied to each, which can be specified using the `suffixes` parameter

> **Trouble About**
>
> You may have noticed that the above solution breaks down if we have two employees with the same name. In this case, we need unique identifiers (UIDs). More on this in WDSS's SQL for Scientists, coming soon.

### Filtering Joins

The joins above are known as mutating joins, as they mutate the structure of the dataframes. Another type of join is the filtering join. This is where we filter one dataframe using a key column based on the presence of values in another dataframe's corresponding key column. 

Whereas languages such as R have dedicated functions for this, Pandas is optimised enough to use `.isin()` and `.unique()`.

An semi-join is where we only include rows with key values that can be found in the other dataset.

In [22]:
# Perform a semi-join
employee[employee.name.isin(sale.name.unique())]

,name,years_exp
0,Ann,4
1,Bob,2


An anti-join is the reverse, keeping only rows that _don't_ match.

In [18]:
# Perform an anti-join
employee[~employee.name.isin(sale.name.unique())]

,name,years_exp
2,Cat,1


## Missing Values

You will have noticed above that join operations can introduce missing values, denoted as `NaN`. These also commonly arise in real-world datasets, either because we are missing data or because data was invalid. It is important to be able to find missing values and handle them appropriately. To learn more about missing values, we will work with the `adult` dataset, which contains information about various US adults and their income level. You can download this dataset from the [session webpage](https://education.wdss.io/python-for-data-science/session-four/). The dataset contains many missing values which are marked with a "?".

In [23]:
# Column names not included in dataset so we add them manually
column_names = (
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
    'income'
)
adult = pd.read_csv('data/adult.csv', names=column_names, na_values='?')
# Drop unnecessary columns
adult = adult.drop(['fnlwgt', 'education'], axis=1)

In [24]:
adult.head()

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39.0,State-gov,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50.0,Self-emp-not-inc,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38.0,Private,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53.0,Private,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28.0,Private,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Finding and Removing Missing Values

First we ask how we can find missing values. We do this using the `isna()` method. This returns a Boolean dataframe the same size as the original, with `True` entries wherever the original data was missing. We can then use our usual aggregations to ask questions about the missing values.

In [26]:
# What proportion of each column is missing?
adult.isna().mean()

age               0.000041
workclass         0.057307
education-num     0.000000
marital-status    0.000000
occupation        0.057512
relationship      0.000000
race              0.000000
sex               0.000000
capital-gain      0.000000
capital-loss      0.000000
hours-per-week    0.000000
native-country    0.017546
income            0.000000
dtype: float64

In [27]:
# Which row has the most missing values? Print it out
r = adult.isna().sum(axis=1).argmax()
adult.iloc[[r]]

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
61,32.0,NaN,4,Married-spouse-absent,NaN,Not-in-family,White,Male,0,0,40,NaN,<=50K


The most aggressive method of handling missing values is to either drop columns or rows with missing values. This can be done using the `dropna` method, specifying the axis to specify if we wish to remove rows (`0`, the default) or columns (`1`). By default, rows/columns with any missing values are removed but by modifying the parameters this can be changed to all missing or a above a certain number. Read more in [the docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html).

In [28]:
# Drop all rows with missing values from `adult` and report the change in size
print("Number of rows before:", adult.shape[0])
adult_dropna = adult.dropna()
print("Number of rows before:", adult_dropna.shape[0])

Number of rows before: 48842
Number of rows before: 45220


In some cases, we know what value a missing value represents. For example, when we performed a left join before.

In [29]:
employeed_sales = pd.merge(employee, sale, how='left', on='name')
employeed_sales

,name,years_exp,day,value
0,Ann,4,Mon,5.0
1,Ann,4,Wed,8.0
2,Ann,4,Fri,8.0
3,Bob,2,Mon,7.0
4,Bob,2,Thu,4.0
5,Cat,1,NaN,NaN


Here, absence from the sales table implies that no sales were made and so we can replace missing values in the `value` column with `0`. We do this using the `fillna()` method. This can be applied to the entire dataframe or just one series. The method can do complex fills such as back-filling and padding, but we will look at the simplest case when we pass in a single number to fill missing values with. You can read about the other options [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html).

In [30]:
# How much did each employee make in sales?
employeed_sales.value.fillna(0)
employeed_sales.groupby('name').value.sum()

name
Ann    21.0
Bob    11.0
Cat     0.0
Name: value, dtype: float64

### Imputation

The two methods of handling missing values above are rather heavy handed. Instead, we may wish to use a predictive approach. This is known as imputation, and is where we guess sensible values of the missing fields. In advanced cases, this may involve using the known features to predict the missing ones, but we'll focus on a simpler case in which we fill missing values based on the non-missing values in the same column.

When dealing with missing values in continuous columns it makes the some sense to fill these values with the column mean or median.

In [33]:
# Fill missing ages with the median age
adult.age = adult.age.fillna(adult.age.median())

For discrete variables, these may give nonsensical results, so it is often better to use the column mode (most common value). We calculate this using the `value_counts()` method before extracting the first index.

In [38]:
# Fill missing countries with modal country
modal_country = adult['native-country'].value_counts().index[0]
print("Modal country:", modal_country)
adult['native-country'] = adult['native-country'].fillna(modal_country)

Modal country: United-States


> For a deeper discussion on how to handle missing values, check out [episode 5](https://youtu.be/BIoFwGl2Vtc?t=1150) of WDSS's podcast [DataBasic](https://podcast.wdss.io/).

## Reshaping Data

It is often the case that we have the correct data but in the wrong shape. In this section we will learn how to reshape data from one form to another. This is an especially important skill if you often work with data stored in Excel, as the free-form nature of spreadsheets can often encourage peculiar data structures.

We do not have the time in this course to go into detail about the pros and cons of various dataframe shapes. Instead, I strongly recommend you watch 4:25–36:42 of [session four](https://youtu.be/_1vSZ1NMgNI?t=265) from WDSS's 'Into the Tidyverse' course. The teaching is in R but the ideas presented about 'tidy' data are language agnostic and of much importance. Pandas does not take the principles of tidy data as strictly as in R's tidyverse, but they are still crucial to be aware of.

### Transposition

The simplest reshaping method is transposition. This involves swapping the columns and rows of our dataset, flipping the entire table along its diagonal. We do this using the `.transpose()` method (which copies by reference). We will create a mock dataframe below in which it would be sensible to perform a transposition.

In [40]:
# Mock data (almost certainly) requiring transposition
weather = pd.DataFrame({
    2016: [16, 1, 0.24],
    2017: [15, 4, 0.21],
    2018: [13, 0, 0.25],
    2019: [26, 2, 0.32],
}, index=['rainfall', 'snowfall', 'cloud_coverage'])
weather

,2016,2017,2018,2019
rainfall,16.00,15.00,13.00,26.00
snowfall,1.00,4.00,0.00,2.00
cloud_coverage,0.24,0.21,0.25,0.32


In [42]:
# Transpose the weather dataset
weather.transpose()

,rainfall,snowfall,cloud_coverage
2016,16.0,1.0,0.24
2017,15.0,4.0,0.21
2018,13.0,0.0,0.25
2019,26.0,2.0,0.32


> **Word of Warning**
>
> Watch out for changing column types when reshaping data. Columns in Pandas must be of one type and so when we switch rows/columns or perform the other transformations that we'll see in a moment, we can have unexpected type coercion, which we may need to correct.

### Pivoting

Pivoting is an operation in Pandas that we perform when we have a column containing variable names or indexes values. We perform a pivot using the `pivot` method, passing in the columns in our current dataframe whose values should be turned into indexes, columns, or stay as values. We'll again work on a mock example.

In [43]:
# Mock data (likely) requiring pivoting
weather = pd.DataFrame({
    'year': [2016, 2016, 2017, 2017, 2018, 2018],
    'variable': ['rainfall', 'snowfall', 'rainfall',
                 'snowfall', 'rainfall', 'snowfall'],
    'value': [16, 1, 15, 4, 13, 0]
})
weather

,year,variable,value
0,2016,rainfall,16
1,2016,snowfall,1
2,2017,rainfall,15
3,2017,snowfall,4
4,2018,rainfall,13
5,2018,snowfall,0


In [44]:
# Pivot the dataframe
weather.pivot(index='year', columns='variable', values='value')

variable,rainfall,snowfall
year,,
2016,16,1
2017,15,4
2018,13,0


> **Good to Know**
>
> We can also pass lists of column names as any arguments of `pivot`

> **Why Can't We All Agree**
>
> Don't confuse Pandas pivoting with:
>
> 1. Excel's pivot tables which are performed using `pd.pivot_table`
> 2. R's pivoting, which includes both pivot (`pivot_wider`/`spread`) and melting (`pivot_longer`/`gather`)

### Melting

The reverse operation of pivoting is melting. This takes multiple columns and turns them into a pair of columns, one containing variable names, the other values. We do this using the `melt` method, passing in a list of variables to be kept the same `id_vars` and (optionally) the name of the new column to create `var_name`. We again practice this on a mock dataset.

In [45]:
# Mock data (possibly) requiring melting
measurements = pd.DataFrame({
    'name': ["Ann", "Bob", "Cat"],
    'height': [157, 172, 168],
    'weight': [10.3, 12.4, 11.2],
    'pace': [6.2, 5.9, 5.4]
})
measurements

,name,height,weight,pace
0,Ann,157,10.3,6.2
1,Bob,172,12.4,5.9
2,Cat,168,11.2,5.4


In [48]:
# Melt the three measurement columns
measurements.melt(id_vars=['name'], var_name='measurement')

,name,measurement,value
0,Ann,height,157.0
1,Bob,height,172.0
2,Cat,height,168.0
3,Ann,weight,10.3
4,Bob,weight,12.4
5,Cat,weight,11.2
6,Ann,pace,6.2
7,Bob,pace,5.9
8,Cat,pace,5.4


> **To Reshape or Not To Reshape**
>
> It can sometimes be hard to know when to reshape data and towards what target. It is first worth noting that the best shape for data is highly contextual, depending on our objectives and whether we are focusing on human or computer readability. In general it is best to follow tidy data practices; as with all rules, however, the sign of an experienced Pandas user is that they know when to break them. This is something you will learn with time and through looking at others' code.

### Stacking

Another important data reshaping technique is stacking and unstacking. This is right on the border between intermediate and advanced techniques, so I have decided to cover it at the start of my office hours this week (a recording of which will be made available), alongside a brief discussion about multi-indexes. 

## Encoding Variables

Computers like numbers. So far we have seen how powerful NumPy is and we will likewise see the same with regard to scikit-learn in session five. These tools however rely on having a single datatype for our data which is most often chosen to be numeric. For that reason, we need to find a way of converting our text variables to numeric forms. This process is known as _encoding_ the variables.

### Dummy Variables

The most common encoding technique for **categorical** text variables is called one-hot encoding. This involves creating a new binary column for every possible value the categorical variable takes, taking value one if and only if the original value corresponds to that column. This is best seen with an example.

![One-hot encoding example](images/one_hot_enc.png)

These new columns of zero and ones are known (particularly in economics) as dummy variables. We can generate these using `pd.get_dummies`, which accepts a series or dataframe.

In [83]:
# Dummify the race column of `adult`
pd.get_dummies(adult.race)

,Amer-Indian-Eskimo,Asian-Pac-Islander,Black,Other,White
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,1,0,0
4,0,0,1,0,0
...,...,...,...,...,...
48837,0,0,0,0,1
48838,0,0,1,0,0
48839,0,0,0,0,1
48840,0,1,0,0,0


Once we have our dummies, we can drop the original column and concatenate our dummies.

Hang on a second though. Let's have a look at the unique values of `adult.race`.

In [84]:
adult.race.unique()

array(['White', 'Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo',
       'Other'], dtype=object)

It's subtle, but there is a possible problem here. Namely, we have redundant information in our model. Every observation has a corresponding race and so if all but one of the dummy columns are zero, the last must be a one. Therefore, we don't actually need all columns but can instead one. In fact, for many algorithms (such as linear regression), this will prevent us from fitting a model. We can avoid this by using `drop_first=True` or by manually dropping a column. In this case, it makes sense for us to drop `Other`.

In [49]:
# Dummify the race column of `adult` and drop `Other`
pd.get_dummies(adult.race).drop('Other', axis=1)

,Amer-Indian-Eskimo,Asian-Pac-Islander,Black,White
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,1,0
4,0,0,1,0
...,...,...,...,...
48837,0,0,0,1
48838,0,0,1,0
48839,0,0,0,1
48840,0,1,0,0


> **Further Reading**
>
> Although one-hot encoding is the most popular encoding method, there are many others available which shine in particular use cases. One other prominent example is ordinal encoding. Learn more about these [here](https://www.analyticsvidhya.com/blog/2020/08/types-of-categorical-data-encoding/), being aware that this includes techniques from `sklearn` which we will learn about in session five.

### Embeddings

One-hot encoding and similar methods work fine for categorical data with only a few levels, but what about when we have a large or virtually infinite number of levels. One approach would be to group various categories together but this is only a temporary solution. A far more powerful solution is to use learned embeddings. An embedding is a way of mapping an input to numeric data space that attempts to capture the structure and relations of the original inputs. These are often learnt using advanced machine learning techniques such as neural networks.

Thankfully, for many cases, trained models are available to us. One example is Word2Vec, which takes English words and maps them to a data space that encodes semantic meaning. This means that similar words are close together and we can form analogies such as `Queen - Woman + Man = King`. Read more about Word2Vec in this [WDSS blog post](https://research.wdss.io/word2vec/).

## Wrapping Up

### Date and Times

Throughout this course, we've paid little specific focus to handling dates and times with Pandas. Unfortunately, we not have time to cover this. It is however vital to be aware of and so we hope to run workshops on the topic eventually. For now, you can read more about the available functionality in [this article](https://pandas.pydata.org/docs/user_guide/timeseries.html).

### Window Functions

Window functions are an advanced feature of Pandas that allow you to "roll" functions over a dataframe. In most cases they are not needed or can be avoid by using loops, but for big data cases, the efficiency they provide is essential. Read more about them [here](https://pandas.pydata.org/docs/user_guide/window.html).

### Styling

Pandas dataframes can look a bit dull. Thankfully, there is a flexible API available for styling dataframes, including highlighting specific observations, columns, and fields. You can even colour-code fields as you would with conditional formatting in Excel. The full details can be found [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html), but here is an example to whet your appetite.

In [60]:
adult.iloc[:5, [0, 2, 8]].style.background_gradient(cmap='viridis')

,age,education-num,capital-gain
0,39.000000,13,2174
1,50.000000,13,0
2,38.000000,9,0
3,53.000000,7,0
4,28.000000,13,0
